<a href="https://colab.research.google.com/github/shirleysheng/crude_oil_predict/blob/master/crude_oil_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

from datetime import datetime as dt, timedelta as td
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [0]:
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import (
LinearDiscriminantAnalysis as LDA,
QuadraticDiscriminantAnalysis as QDA
)
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC, SVC

In [0]:
start_date = dt(2016, 1, 10)
end_date = dt(2017, 12, 31)

In [0]:
ts = pd.read_csv('/content/DATA.csv', index_col=[1])

In [0]:
tslag = ts.iloc[:,1:]


In [0]:
for i in range(0,5):
  tslag['oil_Lag%s' % str(i+1)] = ts['oil'].shift(i+1)
  tslag['gold_Lag%s' % str(i+1)] = ts['gold'].shift(i+1)
  tslag['DowJ_Lag%s' % str(i+1)] = ts['DowJ'].shift(i+1)
  tslag['NAS_Lag%s' % str(i+1)] = ts['NAS'].shift(i+1)
  tslag['US_INDEX_Lag%s' % str(i+1)] = ts['US_INDEX'].shift(i+1)

In [7]:
tslag.head()

,oil,gold,DowJ,NAS,US_INDEX,oil_Lag1,gold_Lag1,DowJ_Lag1,NAS_Lag1,US_INDEX_Lag1,oil_Lag2,gold_Lag2,DowJ_Lag2,NAS_Lag2,US_INDEX_Lag2,oil_Lag3,gold_Lag3,DowJ_Lag3,NAS_Lag3,US_INDEX_Lag3,oil_Lag4,gold_Lag4,DowJ_Lag4,NAS_Lag4,US_INDEX_Lag4,oil_Lag5,gold_Lag5,DowJ_Lag5,NAS_Lag5,US_INDEX_Lag5
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-01-02,52.72,1184.25,17832.990234,4726.810059,91.080002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-05,50.05,1192.00,17501.650391,4652.569824,91.379997,52.72,1184.25,17832.990234,4726.810059,91.080002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-06,47.98,1211.00,17371.640625,4592.740234,91.500000,50.05,1192.00,17501.650391,4652.569824,91.379997,52.72,1184.25,17832.990234,4726.810059,91.080002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-07,48.69,1213.75,17584.519531,4650.470215,91.889999,47.98,1211.00,17371.640625,4592.740234,91.500000,50.05,1192.00,17501.650391,4652.569824,91.379997,52.72,1184.25,17832.990234,4726.810059,91.080002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-08,48.80,1206.50,17907.869141,4736.189941,92.370003,48.69,1213.75,17584.519531,4650.470215,91.889999,47.98,1211.00,17371.640625,4592.740234,91.500000,50.05,1192.00,17501.650391,4652.569824,91.379997,52.72,1184.25,17832.990234,4726.810059,91.080002,NaN,NaN,NaN,NaN,NaN


In [0]:
# Create the returns DataFrame
tsret = tslag.pct_change()*100.0

In [9]:
tsret['oil'].describe()

count    1319.000000
mean       -0.227352
std         9.792551
min      -301.966139
25%        -1.390699
50%         0.062125
75%         1.332600
max        53.086420
Name: oil, dtype: float64

In [0]:
# If any of the values of percentage returns equal zero, set them to
# a small number (stops issues with QDA model in scikit-learn)
tsret.loc[tsret['oil'].abs() < 0.0001, ['oil']] = 0.0001

In [0]:
# Create the "Direction" column (+1 or -1) indicating an up/down day
tsret['Direction'] = np.sign(tsret['oil'])

In [0]:
tsret = tsret.dropna()

In [13]:
set(tsret['Direction'])# examine we have two class -1 for down trend and +1 for up trend

{-1.0, 1.0}

In [0]:
X = tsret.iloc[:,1:20]
y  = tsret.iloc[:,-1]

# The test data is split into two parts: Before and after 1st Jan 2017.
start_test = dt(2020, 1, 1)
# Create training and test sets
X_train = X[pd.to_datetime(X.index) < start_test]
X_test = X[pd.to_datetime(X.index) >= start_test]
y_train = y[pd.to_datetime(y.index) < start_test]
y_test = y[pd.to_datetime(y.index) >= start_test]

In [15]:
X_train.head()

,gold,DowJ,NAS,US_INDEX,oil_Lag1,gold_Lag1,DowJ_Lag1,NAS_Lag1,US_INDEX_Lag1,oil_Lag2,gold_Lag2,DowJ_Lag2,NAS_Lag2,US_INDEX_Lag2,oil_Lag3,gold_Lag3,DowJ_Lag3,NAS_Lag3,US_INDEX_Lag3
date,,,,,,,,,,,,,,,,,,,
2015-01-12,0.887513,-0.544214,-0.836719,0.043508,-0.922131,0.393701,-0.952095,-0.678185,-0.465520,0.225919,-0.597322,1.838831,1.843249,0.522368,1.479783,0.227085,1.225439,1.256983,0.426228
2015-01-13,1.391162,-0.153962,-0.068814,0.358768,-4.736298,0.887513,-0.544214,-0.836719,0.043508,-0.922131,0.393701,-0.952095,-0.678185,-0.465520,0.225919,-0.597322,1.838831,1.843249,0.522368
2015-01-14,-0.827280,-1.059346,-0.475816,-0.162489,-0.303951,1.391162,-0.153962,-0.068814,0.358768,-4.736298,0.887513,-0.544214,-0.836719,0.043508,-0.922131,0.393701,-0.952095,-0.678185,-0.465520
2015-01-15,0.528993,-0.610423,-1.476510,0.206157,5.596690,-0.827280,-1.059346,-0.475816,-0.162489,-0.303951,1.391162,-0.153962,-0.068814,0.358768,-4.736298,0.887513,-0.544214,-0.836719,0.043508
2015-01-16,1.861971,1.101914,1.390561,0.184081,-4.372035,0.528993,-0.610423,-1.476510,0.206157,5.596690,-0.827280,-1.059346,-0.475816,-0.162489,-0.303951,1.391162,-0.153962,-0.068814,0.358768


In [16]:
models = [
  ('LR', LogisticRegression()),
  ('LDA', LDA()),

  ('QDA', QDA()),
  ('LSVC', LinearSVC(max_iter=10000)),

  ('RSVM', SVC(C=1000000.0, cache_size=200, class_weight=None,
    coef0=0.0, degree=3, gamma=0.0001, kernel='rbf',
    max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)),

  ('RF', RandomForestClassifier(n_estimators=1000, criterion='gini',
  max_depth=None, min_samples_split=2,
  min_samples_leaf=1, max_features='auto',
  bootstrap=True, oob_score=False, n_jobs=1,
  random_state=None, verbose=0)
  )
]
# Iterate through the models
for m in models:
# Train each of the models on the training set
  m[1].fit(X_train, y_train)
# Make an array of predictions on the test set
  pred = m[1].predict(X_test)


# Output the accuracy and the confusion matrix for each model
  print('%s:\n%0.3f' % (m[0], m[1].score(X_test, y_test)))
  print('%s\n' % confusion_matrix(pred, y_test))

LR:
0.549
[[29 18]
 [28 27]]

LDA:
0.549
[[29 18]
 [28 27]]

QDA:
0.569
[[33 20]
 [24 25]]

LSVC:
0.549
[[29 18]
 [28 27]]

RSVM:
0.539
[[30 20]
 [27 25]]

RF:
0.500
[[25 19]
 [32 26]]



In [0]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score,recall_score,precision_score

In [18]:
logistic = LogisticRegression()
grid_logi_values = {'penalty': ['l1', 'l2'],'C':np.logspace(-3,3,10)}
grid_logi_acc = GridSearchCV(logistic, param_grid = grid_logi_values,scoring = 'recall')
grid_logi_acc.fit(X_train, y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
       2.15443469e+02, 1.00000000e+03]),
                         'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
         

In [27]:
#Predict values based on new parameters
logi_pred_acc = grid_logi_acc.predict(X_test)

# New Model Evaluation metrics 
print('Accuracy Score : ' + str(accuracy_score(y_test,logi_pred_acc)))


#LDA1(Grid Search) Confusion matrix
confusion_matrix(y_test,logi_pred_acc)


Accuracy Score : 0.5882352941176471


array([[21, 36],
       [ 6, 39]])

LDA

In [0]:
LDA_estimator_1 = LDA(shrinkage='auto')
LDA_parameters_1 = {
    'solver': ('lsqr','eigen'),  #note svd does not run with shrinkage and models using it will be tuned separately
    'n_components': (1,5,1),
                   }
# with GridSearch
grid_lda_acc1 = GridSearchCV(
    estimator=LDA_estimator_1,
    param_grid=LDA_parameters_1,
    scoring = 'accuracy',
    n_jobs = -1,
    cv = 5
)
grid_lda_acc1.fit(X_train, y_train)
lda_pred_acc1 =grid_lda_acc1.predict(X_test)


In [22]:
# New Model Evaluation metrics 
print('Accuracy Score : ' + str(accuracy_score(y_test,lda_pred_acc1)))


#LDA1(Grid Search) Confusion matrix
confusion_matrix(y_test,lda_pred_acc1)

Accuracy Score : 0.5392156862745098


array([[26, 31],
       [16, 29]])

In [0]:
LDA_estimator_2 = LDA(solver='svd', )#note svd does not run with shrinkage and models using it will be tuned separately
LDA_parameters_2 = {
    'n_components': (0,5,1),
    'store_covariance' :(True, False),
                   }
# with GridSearch
grid_lda_acc2 = GridSearchCV(
    estimator=LDA_estimator_2,
    param_grid=LDA_parameters_2,
    scoring = 'accuracy',
    n_jobs = -1,
    cv = 5
)
grid_lda_acc2.fit(X_train, y_train)
lda_pred_acc2 =grid_lda_acc2.predict(X_test)


In [0]:
# New Model Evaluation metrics 
print('Accuracy Score : ' + str(accuracy_score(y_test,lda_pred_acc2)))


#LDA2(Grid Search) Confusion matrix
confusion_matrix(y_test,lda_pred_acc2)

QDA


In [0]:
QDA_estimator = QDA()
QDA_parameters = {
    'reg_param': (0.00001, 0.0001, 0.001,0.01, 0.1), 
    'store_covariance': (True, False),
    'tol': (0.0001, 0.001,0.01, 0.1), 
                   }
# with GridSearch
grid_qda = GridSearchCV(
    estimator=QDA_estimator,
    param_grid=QDA_parameters,
    scoring = 'accuracy',
    n_jobs = -1,
    cv = 5
)
grid_qda.fit(X_train, y_train)
QDA_pred =grid_qda.predict(X_test)

In [0]:
# New Model Evaluation metrics 
print('Accuracy Score : ' + str(accuracy_score(y_test,QDA_pred)))


#QDA(Grid Search) Confusion matrix
confusion_matrix(y_test,QDA_pred)